# Создание бота на основе `SaluteSpeech` API и `GigaChat` API

В этом уроке:

1.   Знакомство с технологиями `Speech2Text` и `Text2Speech`
2.   Подключение и начало работы с `SaluteSpeech` API (сегодня рассмотрим только синхронный режим)
3.   Соединение аудиобата при помощи `SaluteSpeech` и  `GigaChat`
4.   Создание графического интерфейса для чат-бота на основе `Gradio`


Урок вдохнавлен [туториалом](https://github.com/TirendazAcademy/Gradio-Tutorials/blob/main/ChatBot-with-LangChain/Chatbot-App-with-LangChain-and-Gradio.ipynb)

Отметим, что для `Speech2Text` может также исопльзовать локальная модель [`whisper`](https://openai.com/index/whisper/)



<!-- 1. Подключение и начало работы с SaluteSpeech API
<!-- Официальную документацию можно посмотреть [здесь](https://developers.sber.ru/docs/ru/salutespeech/overview). -->

<!-- Чтобы использовать SaluteSpeech API, нужны авторизационные данные из кабинета Studio. -->

В  работе понадобятся
* [токен для SaluteSpeech](https://developers.sber.ru/docs/ru/salutespeech/quick-start/integration-individuals)
* [токен для гигачат](https://developers.sber.ru/docs/ru/gigachat/quickstart/ind-using-api)  




In [ ]:
auth_speech = open('authspeech.txt').read().strip()
auth_giga   = open('authgiga.txt').read().strip()

## Установка необходимых библиотек

- `GigaChat`, `GigaChain`  для работы с языковой моделью
- `Gradio` для графического интерфейса
- `ffmpeg-python` для работы со звуком в `Colab`

Важно отметить, что на данный момент для `SaluteSpeech` есть только [неофициальный клиент](https://github.com/mmua/salute_speech). Поэтому  будем использовать `requests`.

In [ ]:
!pip install -q gradio  ffmpeg-python

In [ ]:
# !pip install -q -U pyaudio

In [ ]:
!pip install -q -U langchain_gigachat

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 1.1.0 requires langchain-core<2.0.0,>=1.1.0, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.


In [ ]:
import requests
import uuid
#
import wave
import numpy as np
import scipy.io
import gradio as gr
from IPython.display import Audio as IPythonAudio

In [ ]:

import langchain
import langchain_gigachat

langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

In [ ]:
from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, AIMessage

## Знакомство с технологиями Speech2Text и Text2Speech SaluteSpeech
### Speech2Text

Рейтинг лучших Speech2Text моделей можно посмотреть [здесь](https://huggingface.co/spaces/hf-audio/open_asr_leaderboard).

### Text2Speech
Рейтинг моделей для синтеза речи можно посмотреть [здесь](https://huggingface.co/spaces/TTS-AGI/TTS-Arena).


Для `SALUTE_SPEECH` нужно использовать следующий скрипт для получения токена сессии с использованием токена `auth_speech`

In [ ]:
def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None


Получим токен доступа.

In [ ]:
response = get_token(auth_speech)
if response != None:
    salute_token = response.json()['access_token']

NameError: name 'uuid' is not defined

### Запись речи.


In [ ]:
def get_audio():
    print('Это шаблон. Замените шаблон примером кода ниже для локального запуска или для колаба')
    return [],[]



<details><summary>КОД ДЛЯ ЗАПУСКА В <code>КОЛАБ</code></summary>

    
в Google Colab, код выполняется на виртуальной машине в облаке, а значит у Python кода нет доступа к микрофону. Чтобы получить доступ к микрофону придется сделать вставку на JS  коде на Python. JS выполняется в браузере, поэтому через браузер сможет подключиться к микрофону.



```python
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Начать запись");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Остановить запись";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Идет сохранение"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr
```


<details><summary>КОД ДЛЯ ЗАПУСКА <code>ЛОКАЛЬНО</code></summary>

```python
import pyaudio

def get_audio(duration=5, frame_rate=44100, channels=1):
    """
    Записывает аудио с микрофона и возвращает данные в формате для scipy.io.wavfile.write
    
    Параметры:
    duration - длительность записи в секундах (по умолчанию 5)
    frame_rate - частота дискретизации (по умолчанию 44100)
    channels - количество каналов (по умолчанию 1 - моно)
    
    Возвращает:
    audio - numpy массив с аудио данными
    sr - частота дискретизации
    """
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    
    # Создаем объект PyAudio
    p = pyaudio.PyAudio()
    
    # Открываем поток для записи
    stream = p.open(
        format=FORMAT,
        channels=channels,
        rate=frame_rate,
        input=True,
        frames_per_buffer=CHUNK
    )
    
    print(f"Запись дилтельностью {duration} секунд  началась...")
    
    frames = []
    
    # Записываем данные порциями по CHUNK байт
    for _ in range(0, int(frame_rate / CHUNK * duration)):
        
        data = stream.read(CHUNK)
        frames.append(data)
    
    print("Запись завершена!")
    
    # Останавливаем и закрываем поток
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # Преобразуем байты в numpy массив
    audio_data = b''.join(frames)
    audio_array = np.frombuffer(audio_data, dtype=np.int16)
    
    # Если стерео, меняем форму массива
    if channels == 2:
        audio_array = audio_array.reshape(-1, 2)
    
    return audio_array, frame_rate
```

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Начать запись");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Остановить запись";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Идет сохранение"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


Вызовем функцию для записи. Результат можно будет прослушать.

In [ ]:
try:
  audio, sr = get_audio()
except:
  print('ОШИБКА ЗАПИСИ')

Сохраненин аудио в `WAV`-файл.

In [ ]:
try:
  scipy.io.wavfile.write('recording.wav', sr, audio)
except:
  print('ОШИБКА сохранения')

In [ ]:
sampling_rate, audio_data = scipy.io.wavfile.read('recording.wav')
sampling_rate

In [ ]:
IPythonAudio(audio_data,rate=sampling_rate)

### РАСПОЗНОВАНИЕ Речи

Следующая функция `stt` производит трнаскрибацию из аудиофайла согласно [документации](https://developers.sber.ru/docs/ru/salutespeech/recognition/recognition-sync) в синхронном режиме.
- доступные форматы распознаваемого файла (WAV, MP3, OGG, FLAC).

Ответ на запрос от API включает в себя список строк распознанной речи и список эмоций для строк распознанной речи.


In [ ]:
def stt(file_path, token):
    # URL для распознавания речи
    url = "https://smartspeech.sber.ru/rest/v1/speech:recognize"

    # Заголовки запроса
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "audio/x-pcm;bit=16;rate=16000"
    }

    # Открытие аудио файла в бинарном режиме
    with open(file_path, "rb") as audio_file:
        audio_data = audio_file.read()

    # Отправка POST запроса
    response = requests.post(url, headers=headers, data=audio_data, verify=False)

    # Обработка ответа
    if response.status_code == 200:
        result = response.json()
        print("Весь ответ API:", result)
        return result["result"]
    else:
        print("Ошибка:", response.status_code, response.text)
        return response.text

распознаем речь

In [ ]:
rec_speech = stt('recording.wav', salute_token)
print(rec_speech)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Весь ответ API: {'result': ['Тестовая проверка записи.'], 'emotions': [{'negative': 0.00015711258, 'neutral': 0.9997843, 'positive': 5.866122e-05}], 'person_identity': {'age': 'age_none', 'gender': 'gender_none', 'age_score': 0, 'gender_score': 0}, 'status': 200}
['Тестовая проверка записи.']


### Синтез речи
Аналогично может быть произведен и синтез речи, давайте проверим как это работает

In [ ]:
def synthesize_speech(text, token, file2save = 'output.wav', format='wav16', voice='Bys_24000'):
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/text"
    }
    params = {
        "format": format,
        "voice": voice
    }
    response = requests.post(url, headers=headers, params=params, data=text.encode(), verify=False)

    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open(file2save, 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")

In [ ]:
synthesize_speech(rec_speech[0], salute_token)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'


Воспроизведем полученный файл.

In [ ]:
audio_file = 'output.wav'
IPythonAudio(audio_file)

## Созадние аудиодиалога с помошью SaluteSpeech API и GigaChat API

Сделаем диалог в формате аудио-вопрос - текст - ответ - аудио-ответ.

Импорт

In [ ]:

import langchain
langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

In [ ]:
from langchain_gigachat.chat_models import GigaChat

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

Создадим класс для работы с аудио-чатботом `AudioChat`. Класс позволит :
* инициализировать модель LLM (`GigaChat`) для формирования пар вопрос-ответ,  
* сохранять и переиспользовать историю общения в формате вопрос от человека (`HumanMessage`)  и ответ от чата (`AIMessage`) - функция `conversation_predict`
* записывать и транскрибировать запрос и синтезировать ответ голосом - `process_audio`

In [ ]:
class AudioChat:
    def __init__(self, auth_giga, salute_token):
        self.auth_giga = auth_giga
        self.salute_token = salute_token
        self.history = []
        self.llm = GigaChat(credentials=auth_giga, verify_ssl_certs=False)

    def conversation_predict(self, message):
        self.history.append(HumanMessage(content=message))
        gpt_response = self.llm(self.history)
        self.history.append(AIMessage(gpt_response.content))
        return gpt_response.content

    def process_audio(self, file2save='recording.wav', output_file='output.wav'):
        audio, sr = get_audio()
        scipy.io.wavfile.write(file2save, sr, audio)
        rec_speech = stt(file2save, self.salute_token)
        answer = self.conversation_predict(rec_speech[0])
        synthesize_speech(answer, self.salute_token, output_file)
        return output_file



Cоздадим объект класса

In [ ]:
voice_chat = AudioChat(auth_giga, salute_token)

Теперь проверим работу для 1 вопроса

In [ ]:
output_file = voice_chat.process_audio()
IPythonAudio(output_file)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Весь ответ API: {'result': ['Проверим.'], 'emotions': [{'negative': 0.0037818472, 'neutral': 0.992875, 'positive': 0.003343143}], 'person_identity': {'age': 'age_none', 'gender': 'gender_none', 'age_score': 0, 'gender_score': 0}, 'status': 200}


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'


И теперь проверка для второго вопроса

In [ ]:
output_file = voice_chat.process_audio()

IPythonAudio('output.wav')

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Весь ответ API: {'result': ['Просто удивительно.'], 'emotions': [{'negative': 0.089146264, 'neutral': 0.9104774, 'positive': 0.00037625863}], 'person_identity': {'age': 'age_none', 'gender': 'gender_none', 'age_score': 0, 'gender_score': 0}, 'status': 200}


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'


Отметим, что аналогично может быть вызван и текстовый режим работы чата

In [ ]:
voice_chat.conversation_predict('привет, как дела')

'Привет! Всё отлично, спасибо. Готов поговорить, помочь разобраться с вопросом или просто поболтать. Как твои дела?'

Проверим историю

In [ ]:
voice_chat.history

[HumanMessage(content='Проверим.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Пиши — проверю.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Просто удивительно.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Нормально удивлён? Или есть конкретная причина удивления? Расскажи подробнее.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='привет, как дела', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Привет! Всё отлично, спасибо. Готов поговорить, помочь разобраться с вопросом или просто поболтать. Как твои дела?', additional_kwargs={}, response_metadata={})]

## Создание интерфейса диалога в `Gradio`


`Gradio` — [это пакет Python с открытым исходным кодом](https://proglib.io/p/rukovodstvo-po-rabote-s-gradio-sozdanie-veb-interfeysa-dlya-modeley-mashinnogo-obucheniya-2023-03-06), который позволяет быстро создавать простые в использовании настраиваемые компоненты пользовательского интерфейса для вашей модели машинного обучения, любого API или даже произвольной функции Python с помощью нескольких строк кода. Вы можете интегрировать графический интерфейс Gradio непосредственно в Jupyter notebook или поделиться им в виде ссылки с кем угодно. например Hugging Face Spaces поддерживает Gradio.

`Gradio` Идеально подходит для демонстрации или тестирования диалоговых систем без необходимости писать HTML/JS.

### Чат-диалог

Для начала давайте попробуем запустить `Gradio`  в самом простом режиме.

gr.ChatInterface автоматически генерирует удобный чат-интерфейс, где пользователь вводит сообщения, а модель (predict_with_langchain) отвечает на них.

In [ ]:
llm = GigaChat(credentials=auth_giga, verify_ssl_certs=False)
def predict_with_langchain(message, history):
    """Функция для текстового чата, совместимая с gr.ChatInterface"""
    history_langchain_format = []
    for human, assistant in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=assistant))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = llm(history_langchain_format)
    return gpt_response.content

In [ ]:
import gradio as gr
voice_chat = AudioChat(auth_giga, salute_token)

# Создание интерфейса
chat_demo = gr.ChatInterface(
    fn=predict_with_langchain,
    title="Текстовый чат-бот",
    description="Задавайте вопросы текстом."
)

chat_demo.launch(share=True, debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2bf0b2a06f08046ff6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2bf0b2a06f08046ff6.gradio.live


## Создание графического интерфейса для бота на основе `Gradio` для задачи `Sepeech2Text`


У Gradio есть компонент для работы с [аудио](https://www.gradio.app/docs/gradio/audio). Он позволяет как записывать звук, так и воспроизводить. Поэтому в нашем интерфейсе будет два таких компонента, но с разными настройками.

__ВАЖНО интерфейс для работы со звуком не всегда стабильно работает!__

__ВАЖНО в `gradio` возможна и реализация `Text2Speech`, но рессурсо-затратно__



Решения задачи `Sepeech2Text` дополним класс `AudioChat` методом `process_audio2text` - который будет возвращать ответ в виде текста



In [ ]:
class Audio2TextChat(AudioChat):
  def process_audio2text(self, audio_input):
        if audio_input is None:
            return ""
        sr, audio = audio_input
        scipy.io.wavfile.write("input.wav", sr, audio)
        rec_speech = stt("input.wav", self.salute_token)
        answer = self.conversation_predict(rec_speech[0])
        return answer

Создадим объект класса

In [ ]:
voice2text_chat = Audio2TextChat(auth_giga, salute_token)

Создадим минимальную форму для `Gradio`. В форме будут:
* заголовок `Голосовой чат`,
* вход для записи микрофона,
* выход в виде текстового поля `Текст ответа`,
* кнопка для обработки запими
* кнопка для того, чтобы задать новый вопрос (история сессии сохраняется).

После того, как вопрос будет обработан его можно будет также прослушать.

__Отметим__, что пример аналогоичного приложения для whisper может быть найден в [официальной документации `gradio`](https://www.gradio.app/guides/real-time-speech-recognition).

In [ ]:

with gr.Blocks() as demo:
    gr.Markdown("## Голосовой чат")
    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], type="numpy", interactive=True)
    with gr.Row():
        text_output = gr.Textbox(label="Текст ответа", interactive=False)
    with gr.Row():
        btn_text = gr.Button("Обработать")
        btn_clear = gr.Button("Новый вопрос")

    btn_text.click(
        fn=voice2text_chat.process_audio2text,
        inputs=audio_input,
        outputs=text_output
    )
    btn_clear.click(
        fn=lambda: (None, ""),
        inputs=[],
        outputs=[audio_input, text_output]
    )

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bf0372dd23bf4dd700.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Весь ответ API: {'result': ['Самое последнее, что я здесь делал, это полностью голосовой чат на radio, но он работает только в колабе. Ну, по крайней мере, у меня вот здесь у меня potm 3 9 греди не запускается.', ''], 'emotions': [{'negative': 7.150919e-05, 'neutral': 0.99989057, 'positive': 3.7914007e-05}, {'negative': 0, 'neutral': 1, 'positive': 0}], 'person_identity': {'age': 'age_none', 'gender': 'gender_none', 'age_score': 0, 'gender_score': 0}, 'status': 200}


In [ ]:
gr.__version__

'5.50.0'

## Упражнение
1. Сделать чат с голосовыми вопросом и ответом.
2. Разобраться в langchain и создать/добавить телеграмм бот с транскрибацией https://docs.langchain.com/oss/python/integrations/document_loaders/telegram